In [1]:
import pandas as pd
import numpy as np
import os
import pandas_datareader as pdr
from datetime import datetime

In [2]:
# set time frame
timeframe = '1D'

# start and end date
start_date = datetime(2004,6,1)
end_date = datetime(2009,6,1)

# set tickers
tickers = ['AMD']

# pulling data from pdr 
tickers_df = pdr.DataReader(tickers,'yahoo',start_date,end_date)

In [3]:
%store tickers

Stored 'tickers' (list)


In [4]:
tickers_df.head()

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,AMD,AMD,AMD,AMD,AMD,AMD
Date,,,,,,
2004-06-01,15.30,15.30,15.60,15.15,15.50,5964000
2004-06-02,15.08,15.08,15.40,14.89,15.35,6467500
2004-06-03,14.59,14.59,14.95,14.58,14.95,7615400
2004-06-04,15.12,15.12,15.25,14.83,14.89,9378400
2004-06-07,15.80,15.80,15.90,15.29,15.30,9807500


In [5]:
# closing prices dataframe
data = pd.DataFrame()

# get closing prices for all tickers in tickers
for ticker in tickers:
    data[ticker]=tickers_df['Adj Close'][ticker]

In [6]:
data.rename(columns={tickers[0]: 'Close'}, inplace=True)

In [7]:
data

,Close
Date,
2004-06-01,15.30
2004-06-02,15.08
2004-06-03,14.59
2004-06-04,15.12
2004-06-07,15.80
...,...
2009-05-26,4.53
2009-05-27,4.71
2009-05-28,4.70


In [8]:
# add percent change
data['daily_return'] = data['Close'].pct_change().dropna()
data['volume'] = tickers_df['Volume'][ticker]

In [9]:
data = data.dropna()
data.head()

,Close,daily_return,volume
Date,,,
2004-06-02,15.08,-0.014379,6467500
2004-06-03,14.59,-0.032493,7615400
2004-06-04,15.12,0.036326,9378400
2004-06-07,15.80,0.044974,9807500
2004-06-08,16.18,0.024051,9404200


In [10]:
# EWMA Short and Long Windows
short_window = 5
long_window = 50

# Fast and Slow EWMAs from short and long windows
data['ewma_short'] = data['Close'].ewm(halflife=short_window).mean()
data['ewma_long'] = data['Close'].ewm(halflife=long_window).mean()

# Construct a crossover trading signal
data['crossover_long'] = np.where(data['ewma_short'] > data['ewma_long'], 1.0, 0.0) 

# making sep columns if (where) fast passes close == long 
data['crossover_short'] = np.where(data['ewma_short'] < data['ewma_long'], -1.0, 0.0)
data['crossover_signal'] = data['crossover_long'] + data['crossover_short']


<ipython-input-10-06876fc3a873>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ewma_short'] = data['Close'].ewm(halflife=short_window).mean()
<ipython-input-10-06876fc3a873>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ewma_long'] = data['Close'].ewm(halflife=long_window).mean()
<ipython-input-10-06876fc3a873>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [11]:
# Volatility - Short and Long Windows
short_vol_window = 5
long_vol_window = 50

# Construct a `Fast` and `Slow` Exponential Moving Average from short and long windows, respectively
data['short_vol'] = data['daily_return'].ewm(halflife=short_vol_window).std()
data['long_vol'] = data['daily_return'].ewm(halflife=long_vol_window).std()

# Construct a crossover trading signal
data['vol_trend_long'] = np.where(data['short_vol'] < data['long_vol'], 1.0, 0.0)
data['vol_trend_short'] = np.where(data['short_vol'] > data['long_vol'], -1.0, 0.0) 
data['vol_trend_signal'] = data['vol_trend_long'] + data['vol_trend_short']


<ipython-input-11-508068c14cbe>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['short_vol'] = data['daily_return'].ewm(halflife=short_vol_window).std()
<ipython-input-11-508068c14cbe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['long_vol'] = data['daily_return'].ewm(halflife=long_vol_window).std()
<ipython-input-11-508068c14cbe>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [12]:
data = data.dropna()

In [13]:
# Set bollinger band window
bollinger_window = 20

# Calculate rolling mean and standard deviation
data['bollinger_mid_band'] = data['Close'].rolling(window=bollinger_window).mean()
data['bollinger_std'] = data['Close'].rolling(window=20).std() # will use for upper and lower bands of bollinger

# Calculate upper and lowers bands of bollinger band
data['bollinger_upper_band']  = data['bollinger_mid_band'] + (data['bollinger_std'] * 1)
data['bollinger_lower_band']  = data['bollinger_mid_band'] - (data['bollinger_std'] * 1)

# Calculate bollinger band trading signal # signal to calc (doing mean revering strategy)
data['bollinger_long'] = np.where(data['Close'] < data['bollinger_lower_band'], 1.0, 0.0)
data['bollinger_short'] = np.where(data['Close'] > data['bollinger_upper_band'], -1.0, 0.0)
data['bollinger_signal'] = data['bollinger_long'] + data['bollinger_short']

In [14]:
#Calculate simple moving average of volume
data['Short_Volume'] = data['volume'].transform(lambda x: x.rolling(window = 5).mean())
data['Long_Volume'] = data['volume'].transform(lambda x: x.rolling(window = 15).mean())
data['SMA_Volume_Ratio'] = data['Short_Volume']/data['Long_Volume']

# Construct a crossover trading signal
data['crossover_vol_long'] = np.where(data['Short_Volume'] > data['Long_Volume'], 1.0, 0.0) 

# making sep columns if (where) fast passes close == long 
data['crossover_vol_short'] = np.where(data['Short_Volume'] < data['Long_Volume'], -1.0, 0.0)
data['crossover_vol_signal'] = data['crossover_vol_long'] + data['crossover_vol_short']

In [15]:
#Calculate the average true range and keep it in a function
def add_atr_to_dataframe (dataframe, newdf):
    newdf['ATR1'] = abs (dataframe['High'][ticker] - dataframe['Low'][ticker])
    newdf['ATR2'] = abs (dataframe['High'][ticker] - dataframe['Close'][ticker].shift())
    newdf['ATR3'] = abs (dataframe['Low'][ticker] - dataframe['Close'][ticker].shift())
    newdf['TrueRange'] = newdf[['ATR1', 'ATR2', 'ATR3']].max(axis=1)
    return newdf

#Call the average true range function
add_atr_to_dataframe(tickers_df, data)

,Close,daily_return,volume,ewma_short,ewma_long,crossover_long,crossover_short,crossover_signal,short_vol,long_vol,...,Short_Volume,Long_Volume,SMA_Volume_Ratio,crossover_vol_long,crossover_vol_short,crossover_vol_signal,ATR1,ATR2,ATR3,TrueRange
Date,,,,,,,,,,,,,,,,,,,,,
2004-06-03,14.59,-0.032493,7615400,14.818045,14.833302,0.0,-1.0,-1.0,0.012809,0.012809,...,NaN,NaN,NaN,0.0,0.0,0.0,0.370000,0.13,0.500000,0.500000
2004-06-04,15.12,0.036326,9378400,14.932926,14.930196,1.0,0.0,1.0,0.037068,0.035815,...,NaN,NaN,NaN,0.0,0.0,0.0,0.420000,0.66,0.240000,0.660000
2004-06-07,15.80,0.044974,9807500,15.196622,15.152189,1.0,0.0,1.0,0.038141,0.037939,...,NaN,NaN,NaN,0.0,0.0,0.0,0.610000,0.78,0.170000,0.780000
2004-06-08,16.18,0.024051,9404200,15.451217,15.363490,1.0,0.0,1.0,0.032285,0.033437,...,NaN,NaN,NaN,0.0,0.0,0.0,0.570001,0.39,0.180000,0.570001
2004-06-09,15.51,-0.041409,14821000,15.464692,15.388762,1.0,0.0,1.0,0.038395,0.037136,...,10205300.0,NaN,NaN,0.0,0.0,0.0,0.870001,0.17,0.700001,0.870001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-05-26,4.53,0.063380,16094300,4.222073,3.473992,1.0,0.0,1.0,0.045004,0.057221,...,17454180.0,2.204778e+07,0.791652,0.0,-1.0,-1.0,0.340000,0.31,0.030000,0.340000
2009-05-27,4.71,0.039735,21512600,4.285235,3.491009,1.0,0.0,1.0,0.043095,0.056960,...,17117500.0,2.239063e+07,0.764494,0.0,-1.0,-1.0,0.250000,0.27,0.020000,0.270000
2009-05-28,4.70,-0.002123,18383900,4.338926,3.507653,1.0,0.0,1.0,0.040673,0.056576,...,15751460.0,2.261113e+07,0.696624,0.0,-1.0,-1.0,0.300000,0.13,0.170000,0.300000


In [16]:
# Construct a crossover trading signal
data['crossover_atr_entry'] = np.where(data['TrueRange'] < 0.7, 1.0, 0.0) 

# making sep columns if (where) fast passes close == long 
data['crossover_atr_exit'] = np.where(data['TrueRange'] > 0.9, -1.0, 0.0)
data['crossover_atr_signal'] = data['crossover_atr_entry'] + data['crossover_atr_exit']

data

,Close,daily_return,volume,ewma_short,ewma_long,crossover_long,crossover_short,crossover_signal,short_vol,long_vol,...,crossover_vol_long,crossover_vol_short,crossover_vol_signal,ATR1,ATR2,ATR3,TrueRange,crossover_atr_entry,crossover_atr_exit,crossover_atr_signal
Date,,,,,,,,,,,,,,,,,,,,,
2004-06-03,14.59,-0.032493,7615400,14.818045,14.833302,0.0,-1.0,-1.0,0.012809,0.012809,...,0.0,0.0,0.0,0.370000,0.13,0.500000,0.500000,1.0,0.0,1.0
2004-06-04,15.12,0.036326,9378400,14.932926,14.930196,1.0,0.0,1.0,0.037068,0.035815,...,0.0,0.0,0.0,0.420000,0.66,0.240000,0.660000,1.0,0.0,1.0
2004-06-07,15.80,0.044974,9807500,15.196622,15.152189,1.0,0.0,1.0,0.038141,0.037939,...,0.0,0.0,0.0,0.610000,0.78,0.170000,0.780000,0.0,0.0,0.0
2004-06-08,16.18,0.024051,9404200,15.451217,15.363490,1.0,0.0,1.0,0.032285,0.033437,...,0.0,0.0,0.0,0.570001,0.39,0.180000,0.570001,1.0,0.0,1.0
2004-06-09,15.51,-0.041409,14821000,15.464692,15.388762,1.0,0.0,1.0,0.038395,0.037136,...,0.0,0.0,0.0,0.870001,0.17,0.700001,0.870001,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-05-26,4.53,0.063380,16094300,4.222073,3.473992,1.0,0.0,1.0,0.045004,0.057221,...,0.0,-1.0,-1.0,0.340000,0.31,0.030000,0.340000,1.0,0.0,1.0
2009-05-27,4.71,0.039735,21512600,4.285235,3.491009,1.0,0.0,1.0,0.043095,0.056960,...,0.0,-1.0,-1.0,0.250000,0.27,0.020000,0.270000,1.0,0.0,1.0
2009-05-28,4.70,-0.002123,18383900,4.338926,3.507653,1.0,0.0,1.0,0.040673,0.056576,...,0.0,-1.0,-1.0,0.300000,0.13,0.170000,0.300000,1.0,0.0,1.0


In [17]:
data.to_csv("../Data/" + str(tickers[0]) +  "_Closing_2008" +  ".csv",index=True)